# 509 Final Project

## Rsolve dependencies

## Globally import libraries

In [1]:
import numpy as np
import pandas as pd
import pymysql as mysql
import matplotlib.pyplot as plt
import os
import shutil
import re
import logging
import time
import datetime as dt
import zipfile
import requests
from bs4 import BeautifulSoup
import datetime
import re
import regex as rex
from collections import defaultdict, Counter
import random
import json
#import mysql.connector

# Set pandas global options
pd.options.display.max_rows = 17

## Upload data from CSV

In [2]:
'''Dir nav citation:
https://softhints.com/python-change-directory-parent/'''
curr_dir = os.path.abspath(os.curdir)
print(curr_dir)
os.chdir("..")
up1_dir = os.path.abspath(os.curdir)
print(up1_dir)

C:\Users\acarr\Documents\GitHub\ADS509_Final_project\deliverables
C:\Users\acarr\Documents\GitHub\ADS509_Final_project


In [3]:
# change `data_location` to the location of the folder on your machine.
data_r_location = 'data_restricted'
data_location = 'data'

file_in_name = 'data_raw_amc.csv'
file_out_name = 'data_parsed_amc.csv'

file_in_path = os.path.join(up1_dir, data_r_location, file_in_name)
file_out_path = os.path.join(up1_dir, data_location, file_out_name)

print(f'CSV file in path: {file_in_path}')
print(f'CSV file out path: {file_out_path}')

CSV file in path: C:\Users\acarr\Documents\GitHub\ADS509_Final_project\data_restricted\data_raw_amc.csv
CSV file out path: C:\Users\acarr\Documents\GitHub\ADS509_Final_project\data\data_parsed_amc.csv


In [4]:
slct_tbl_full_df01 = pd.read_csv(file_in_path)
display(slct_tbl_full_df01.head())

,text_id,source_name,author,title,url,publish_date,article_text,content
0,1,CNN,"Clare Foran,Nicky Robertson",Senate races to avert default but vote timing ...,https://www.cnn.com/2023/06/01/politics/senate...,2023-06-01T09:00:40Z,"<!DOCTYPE html>\n<html data-layout-uri=""cms.cn...",NaN
1,2,Fox News,Paul Steinhauser,First on Fox: Pro-Tim Scott super PAC launches...,https://www.foxnews.com/politics/pro-scott-sup...,2023-06-01T16:12:56Z,"<!DOCTYPE html>\n<html data-n-head=""%7B%22lang...",NaN
2,3,Fox News,Greg Wehner,Pennsylvania bus driver allegedly used duct ta...,https://www.foxnews.com/us/pennsylvania-bus-dr...,2023-05-31T00:21:20Z,"<!DOCTYPE html>\n<html data-n-head=""%7B%22lang...",NaN
3,4,Fox News,Michael Ruiz,Bob Lee murder: Cash App founder seen with sus...,https://www.foxnews.com/us/bob-lee-murder-cash...,2023-05-31T20:30:12Z,"<!DOCTYPE html>\n<html data-n-head=""%7B%22lang...",NaN
4,5,Fox News,Brooke Singman,Senate GOP demands answers on security clearan...,https://www.foxnews.com/politics/senate-gop-de...,2023-05-31T22:11:38Z,"<!DOCTYPE html>\n<html data-n-head=""%7B%22lang...",NaN


In [5]:
count_nan = slct_tbl_full_df01['article_text'].isnull().sum()
 
# printing the number of values present
# in the column
print('Number of NaN values present: ' + str(count_nan))

Number of NaN values present: 0


In [6]:
slct_tbl_full_df02 = slct_tbl_full_df01.copy()
#slct_tbl_full_df02 = slct_tbl_full_df01.sort_values(by=['source_name'])
#slct_tbl_full_df02 = slct_tbl_full_df02.reset_index()

print(f'DF instances: {len(slct_tbl_full_df02)}')

slct_tbl_full_df02['article_parsed'] = ''

total_urls = len(slct_tbl_full_df02)

# Start timer
start_time = dt.datetime.today()

for i, row in enumerate(slct_tbl_full_df02.itertuples(), 1):
    #print(f'Enumeration #: {i}')
    #print(row[7])
    soup = BeautifulSoup(row[7], 'html.parser')

    try:
        script_tag = soup.find('script', {'type': 'application/ld+json'})
        if script_tag == None:
            print('\nJSON Object == None: Index:', i-1, '; source:',
                  row[2],
                  '; URL:', row[5])
        else:
            article_json = json.loads(script_tag.string)
            article_content = article_json['articleBody']
            slct_tbl_full_df02.at[row.Index, 'article_parsed'] = article_content

    except KeyError:
        print('\nMissing JSON key: Index:', i-1, '; source:',
              row[2],
              '; URL:', row[5])

    print('.', end='')
        
# End timer script
end_time = dt.datetime.today()
time_elapse = end_time - start_time
print(f'Start Time = {start_time}')
print(f'End Time = {end_time}')
print(f'Elapsed Time = {time_elapse}')

DF instances: 2929
................................................................
Missing JSON key: Index: 64 ; source: CNN ; URL: https://www.cnn.com/politics/live-news/us-debt-ceiling-deadline-talks-05-31-23/index.html
..........
JSON Object == None: Index: 74 ; source: Fox News ; URL: https://press.foxnews.com/2023/06/fox-news-channel-to-debut-new-weekend-primetime-lineup-on-june-3
.............
Missing JSON key: Index: 87 ; source: CNN ; URL: https://www.cnn.com/politics/live-news/us-debt-ceiling-senate-vote-06-01-23/index.html
........................................................................................................................................................................
JSON Object == None: Index: 255 ; source: CNN ; URL: https://view.newsletters.cnn.com/messages/1685581226387e748f0e9a1e2/raw
........................................................................................................................................................
Missing JSON 

In [7]:
display(slct_tbl_full_df02.iloc[2924])

text_id                                                        4780
source_name                                          Breitbart News
author                                     Lucas Nolan, Lucas Nolan
title             'The Lifestyle:' Murdered Cash App Creator Bob...
url               https://www.breitbart.com/tech/2023/05/16/the-...
publish_date                                   2023-05-16T14:46:43Z
article_text      <!DOCTYPE html>\n<html class="post-tmpl-defaul...
content           The brutal murder of successful tech executive...
article_parsed                                                     
Name: 2924, dtype: object

In [14]:
display(slct_tbl_full_df02[slct_tbl_full_df02['article_parsed']==''].head())
print(len(slct_tbl_full_df02[slct_tbl_full_df02['article_parsed']=='']))

,text_id,source_name,author,title,url,publish_date,article_text,content,article_parsed
64,65,CNN,"By <a href=""/profiles/adrienne-vogt"">Adrienne ...",The latest on the US debt ceiling deal,https://www.cnn.com/politics/live-news/us-debt...,2023-05-31T12:35:50Z,"<!DOCTYPE html>\n<html lang=""en"">\n <head>\n ...",NaN,
74,75,Fox News,Sofie Watson,FOX NEWS CHANNEL TO DEBUT NEW WEEKEND PRIMETIM...,https://press.foxnews.com/2023/06/fox-news-cha...,2023-06-01T18:33:43Z,"<!DOCTYPE html>\n<html lang=""en"">\n <head>\n ...",NaN,
87,88,CNN,"By <a href=""/profiles/maureen-chowdhury"">Maure...",The latest on the US debt ceiling deal,https://www.cnn.com/politics/live-news/us-debt...,2023-06-01T13:46:13Z,"<!DOCTYPE html>\n<html lang=""en"">\n <head>\n ...",NaN,
255,257,CNN,NaN,UPDATE: WAR ON GAY PRIDE...,https://view.newsletters.cnn.com/messages/1685...,2023-06-01T21:55:37Z,"<!DOCTYPE html>\n<html lang=""en"" xmlns:o=""urn:...",NaN,
407,566,CNN,"By Mike Hayes and <a href=""/profiles/maureen-c...",The latest on the US debt ceiling deal,https://www.cnn.com/politics/live-news/us-debt...,2023-05-30T14:12:09Z,"<!DOCTYPE html>\n<html lang=""en"">\n <head>\n ...",NaN,


598


In [10]:
count_p_nan = slct_tbl_full_df02['article_parsed'].isnull().sum()
 
# printing the number of values present
# in the column
print('Number of NaN values present: ' + str(count_p_nan))

Number of NaN values present: 0


story_str000 = slct_tbl_full_df01['article_text'][0]
#print(story_str000)

story_str000_bs = BeautifulSoup(story_str000, 'html.parser')
rex_art = rex.compile(r'articleBody[\S\s]*\",')
rex_art.findall(story_str000)

story_str000_bs.find(class_='articleBody')

story_str100 = slct_tbl_full_df01['article_text'][1]
#print(story_str100)

rex_art100 = rex.compile(r'articleBody[\S\s]+}')
rex_art100.findall(story_str100)

In [8]:
slct_tbl_full_df03 = slct_tbl_full_df02[['source_name',
                                                           'author',
                                                           'title',
                                                           'url',
                                                           'publish_date',
                                                           'article_parsed']]

In [9]:
slct_tbl_full_df03.to_csv(file_out_path, index=False)

print(df.head())
print(df['article_text'][100])